In [25]:
import numpy as np
from math import log

# pretty print for np.array 
# from https://stackoverflow.com/questions/53126305/pretty-printing-numpy-ndarrays-using-unicode-characters/53164538#53164538
def pretty_print(A):
    if A.ndim==1:
        print(A)
    else:
        w = max([len(str(s)) for s in A]) 
        print(u'\u250c' + u'\u2500' * w + u'\u2510') 
        for AA in A:
            print(' ', end='')
            print('[', end='')
            for i, AAA in enumerate(AA[:-1]):
                w1 = max([len(str(s)) for s in A[:, i]])
                print(str(AAA) + ' ' * (w1  - len(str(AAA)) + 1), end='')
            w1 = max([len(str(s)) for s in A[:, -1]])
            print(str(AA[-1]) + ' ' * (w1 - len(str(AA[-1]))), end='')
            print(']')
        print(u'\u2514'+u'\u2500' * w + u'\u2518')  

def compare(left: set, right: set):
    if len(left) == len(right):
        for i in range(len(left)):
            if left[i] != right[i]:
                return left[i] < right[i]
    return left < right


### Generate $\mathcal{A}_n^2$

> Let $|\cdot|$ denotes cardinality and $\Delta$ denote symmetric different

In [26]:
def Delta(left: set, right: set):
    return left.symmetric_difference(right)

def Cardi(x: set):
    return len(x)

def legal(alpha_i: set, alpha_i_1: set):
    return Cardi(alpha_i) <= Cardi(alpha_i_1) and len(Delta(alpha_i, alpha_i_1)) <= 2

> Let $n=2^m$, $\Omega$ be a set of $m$ element such that $|\alpha_i| \leq |\alpha_{i+1}|$ and $|\alpha_i \Delta \alpha_{i+1}| \leq 2$  
> Let $\alpha_0=\{\varnothing\}$, now genereate $\Omega$

In [44]:
from itertools import chain, combinations

# param
m = 4
# generate \Omega

def create_Omega(_m):
    s = list(range(1, _m + 1))
    container = chain.from_iterable(combinations(s, r) for r in range(len(s)+1))
    map = {i: [] for i in range(1, len(s) + 1)}
    for item in container:
        if len(item) != 0:
            map[len(item)].append(set(item))
    res = [set(), set()]
    for key, value in map.items():
        if key % 2 == 0:
            value.reverse()
        res.extend(value)
    return res

def assert_omega(ome, _m):
    for i in range(1, _m):
        assert(legal(ome[i], ome[i + 1]))
    assert(len(ome) == 2**_m + 1)

omega = create_Omega(m)
assert_omega(omega, m)
omega

[set(),
 set(),
 {1},
 {2},
 {3},
 {4},
 {3, 4},
 {2, 4},
 {2, 3},
 {1, 4},
 {1, 3},
 {1, 2},
 {1, 2, 3},
 {1, 2, 4},
 {1, 3, 4},
 {2, 3, 4},
 {1, 2, 3, 4}]

> $$a_{ij} = \begin{cases}
    -1,\;\alpha_j\bigcap(\alpha_{i-1}\bigcup\alpha_i)=\alpha_{i-1}\Delta\alpha_{i}\;and\;|\alpha_{i-1}\Delta\alpha_{i}|=2 \\
    (-1)^{|\alpha_{i-1}\bigcap\alpha_j| + 1},\;\alpha_{j}\bigcap(\alpha_{i-1}\bigcup\alpha_{i})\neq\varnothing\;but\;does\;not\;meet\;the\;condition\;above \\
    1,\;\alpha_j\bigcap(\alpha_{i-1}\bigcup\alpha_i)=\varnothing \\
\end{cases}
$$

In [45]:
def query_element(i: int, j: int, _omega: list) -> int:
    alpha_j = _omega[j + 1]
    alpha_i_1 = _omega[i]
    alpha_i = _omega[i + 1]

    if alpha_j.intersection(alpha_i_1.union(alpha_i)) == Delta(alpha_i_1, alpha_i) \
        and Cardi(Delta(alpha_i_1, alpha_i)) == 2:
        return -1
    elif Cardi(alpha_j.intersection(alpha_i_1.union(alpha_i))) != 0:
        return (-1)**(Cardi(alpha_i_1.intersection(alpha_j)) + 1)
    elif Cardi(alpha_j.intersection(alpha_i_1.union(alpha_i))) == 0:
        return 1
    else:
        raise ValueError("Undefined behavior!")
    
def create_A(_omega, _m):
    A_mat = np.zeros((2**_m, 2**_m))
    for i in range(A_mat.shape[0]):
        for j in range(A_mat.shape[1]):
            A_mat[i, j] = query_element(i, j, _omega)
    return A_mat

A_mat = create_A(omega, m)
pretty_print(A_mat.astype(int))

┌─────────────────────────────────────────────────┐
 [1 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 ]
 [1 -1 1  1  1  1  1  1  -1 -1 -1 -1 -1 -1 1  -1]
 [1 1  -1 1  1  1  -1 -1 1  1  -1 -1 -1 1  -1 -1]
 [1 1  1  -1 1  -1 1  -1 1  -1 1  -1 1  -1 -1 -1]
 [1 1  1  1  -1 -1 -1 1  -1 1  1  1  -1 -1 -1 -1]
 [1 1  1  -1 1  1  1  -1 1  -1 1  -1 1  1  1  1 ]
 [1 1  -1 1  1  -1 1  -1 1  1  -1 -1 1  -1 -1 -1]
 [1 1  1  -1 1  -1 -1 1  1  -1 1  1  -1 -1 -1 -1]
 [1 -1 1  1  -1 1  1  -1 -1 1  1  -1 1  1  -1 -1]
 [1 1  1  -1 1  -1 1  -1 -1 1  1  1  -1 -1 -1 -1]
 [1 1  -1 1  1  1  -1 -1 1  -1 1  -1 1  -1 -1 -1]
 [1 1  1  -1 1  -1 1  1  1  1  -1 -1 -1 1  1  -1]
 [1 1  1  1  -1 -1 1  -1 1  -1 -1 1  -1 -1 -1 1 ]
 [1 1  1  -1 1  1  -1 -1 -1 1  -1 -1 1  -1 -1 1 ]
 [1 1  -1 1  1  -1 1  1  -1 -1 -1 -1 -1 1  -1 1 ]
 [1 -1 1  1  1  -1 -1 -1 1  1  1  -1 -1 -1 1  1 ]
└─────────────────────────────────────────────────┘


### Check $A = LQ$

> Let $Q$ be a $n$ by $n$ matrix given by $q_{ij} = (-1)^{|\alpha_i \bigcap \alpha_j|}$, $Q$ is a symmetric Hadamard matrix, that is $Q^2=QQ^T=nI_n$

In [47]:
def get_Q(_omega, _n):
    Q_mat = np.zeros((_n, _n))
    for i in range(_n):
        for j in range(_n):
            Q_mat[i, j] = (-1) ** Cardi(_omega[i + 1].intersection(_omega[j + 1]))
    return np.matrix(Q_mat)

Q_mat = get_Q(omega, 2**m)
pretty_print(np.array((Q_mat * Q_mat).astype(int)))

┌─────────────────────────────────────────────────┐
 [16 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 ]
 [0  16 0  0  0  0  0  0  0  0  0  0  0  0  0  0 ]
 [0  0  16 0  0  0  0  0  0  0  0  0  0  0  0  0 ]
 [0  0  0  16 0  0  0  0  0  0  0  0  0  0  0  0 ]
 [0  0  0  0  16 0  0  0  0  0  0  0  0  0  0  0 ]
 [0  0  0  0  0  16 0  0  0  0  0  0  0  0  0  0 ]
 [0  0  0  0  0  0  16 0  0  0  0  0  0  0  0  0 ]
 [0  0  0  0  0  0  0  16 0  0  0  0  0  0  0  0 ]
 [0  0  0  0  0  0  0  0  16 0  0  0  0  0  0  0 ]
 [0  0  0  0  0  0  0  0  0  16 0  0  0  0  0  0 ]
 [0  0  0  0  0  0  0  0  0  0  16 0  0  0  0  0 ]
 [0  0  0  0  0  0  0  0  0  0  0  16 0  0  0  0 ]
 [0  0  0  0  0  0  0  0  0  0  0  0  16 0  0  0 ]
 [0  0  0  0  0  0  0  0  0  0  0  0  0  16 0  0 ]
 [0  0  0  0  0  0  0  0  0  0  0  0  0  0  16 0 ]
 [0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  16]
└─────────────────────────────────────────────────┘


> Construct matrix $L$ row by row:
> 

In [48]:
pretty_print(np.array(np.matrix(A_mat) * np.matrix(Q_mat).I))

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
 [1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   0.0   1.0   0.0   0.0   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   0.0   0.0   0.0   1.0   0.0   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.5   0.0   0.0   0.5   -0.5  0.5   0.0   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   0.0   0.5   0.5   0.0   -0.5  0.5   0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.0   0.0   0.0   0.5   0.5   0.0   -0.5  0.5    0.0   0.0   0.0   0.0   0.0   0.0   0.0    0.0  ]
 [0.125 0.125 0.125 0.125 0.125 0.125 0.125 -0.875 0.125 0.125 0.125 0.125 

### Generate $\mathcal{A}_{n - 1}^1$

> Let $B\in\mathcal{A}_{n - 1}^1$, $\Phi(B)\in\mathcal{A}_{n}^2\;$:
> $$ \Phi(B) = \left(\begin{array}{cc} 
1 & 1_{n-1}\\
-1^T_{n-1} & 2B-J_{n-1}
\end{array}\right)
> $$
> And: 
> $$ \Phi(B)^{-1} = \left(\begin{array}{cc} 
1-\frac{1}{2}1_{n-1}B^{-1}1^T_{n-1} & -\frac{1}{2}1_{n-1}B^{-1}\\
\frac{1}{2}B^{-1}1^T_{n-1} & \frac{1}{2}B^{-1}
\end{array}\right)
> $$

In [5]:
def create_A_1_mat(A_2_mat):
    A_1_mat = A_2_mat.copy()
    for i in range(1, A_1_mat.shape[0]):
        A_1_mat[i, :] = A_1_mat[i, :] * -1

    for j in range(1, A_1_mat.shape[1]):
        if A_1_mat[0, j] == -1:
            A_1_mat[:, j] = A_1_mat[:, j] * -1

    A_1_mat = (A_1_mat[1:, 1:] + np.ones(2**m - 1)) * 0.5
    assert(A_1_mat.max() == 1 and A_1_mat.min() == 0)
    return A_1_mat

pretty_print(create_A_1_mat(A_mat).astype(int))

┌───────────────────────────────┐
 [1 0 0 0 1 1 1 0 0 0 1 1 1 0 0]
 [0 1 0 0 1 0 0 1 1 0 1 1 0 1 0]
 [0 0 1 0 0 1 0 1 0 1 1 0 1 1 0]
 [0 0 0 1 0 0 1 0 1 1 0 1 1 1 0]
 [0 0 1 0 1 1 0 1 0 1 1 0 0 0 1]
 [0 1 0 0 0 1 0 1 1 0 1 0 1 1 0]
 [0 0 1 0 0 0 1 1 0 1 0 1 1 1 0]
 [1 0 0 1 1 1 0 0 1 1 1 0 0 1 0]
 [0 0 1 0 0 1 0 0 1 1 0 1 1 1 0]
 [0 1 0 0 1 0 0 1 0 1 1 0 1 1 0]
 [0 0 1 0 0 1 1 1 1 0 1 1 0 0 0]
 [0 0 0 1 0 1 0 1 0 0 0 1 1 1 1]
 [0 0 1 0 1 0 0 0 1 0 1 0 1 1 1]
 [0 1 0 0 0 1 1 0 0 0 1 1 0 1 1]
 [1 0 0 0 0 0 0 1 1 1 1 1 1 0 1]
└───────────────────────────────┘


In [6]:
def funcX(A):
    return np.absolute(np.matrix(A).I).max()

def get_o_1(_X, _n):
    return (0.5 * _n * log(_n, 2) - _n - log(_X, 2)) / _n

### Verify $χ(A)=2^{\frac{1}{2}nlogn - n(1+o(1))}$

In [13]:
for m in range(2, 12):
    omega = create_Omega(m)
    assert_omega(omega, m)
    A_2_mat = create_A(omega, m)
    X_A_2 = funcX(A_2_mat)
    n = A_2_mat.shape[0]
    o_1 = get_o_1(X_A_2, n)
    print("m = {m_term}, n = {n_term}, χ(A_2) = {X_A_2_term}, o(1) term = {o_1_term:.3f}".format(m_term=m, n_term=n, X_A_2_term=X_A_2, o_1_term=o_1))

m = 2, n = 4, χ(A_2) = 0.5, o(1) term = 0.250
m = 3, n = 8, χ(A_2) = 1.5, o(1) term = 0.427
m = 4, n = 16, χ(A_2) = 3.223140495867768, o(1) term = 0.894
m = 5, n = 32, χ(A_2) = 12.416009230529388, o(1) term = 1.386
m = 6, n = 64, χ(A_2) = 1.657924391640421, o(1) term = 1.989
m = 7, n = 128, χ(A_2) = 1.6608503312087368, o(1) term = 2.494
m = 8, n = 256, χ(A_2) = 2.0162840881764144, o(1) term = 2.996
m = 9, n = 512, χ(A_2) = 4.744966510402994, o(1) term = 3.496
m = 10, n = 1024, χ(A_2) = 62.67192910411171, o(1) term = 3.994
m = 11, n = 2048, χ(A_2) = 17.43631264284615, o(1) term = 4.498


### Verify $χ(A')=2^{\frac{1}{2}nlogn - n(1+o(1))}$

In [14]:
for m in range(2, 12):
    omega = create_Omega(m)
    assert_omega(omega, m)
    A_mat = create_A(omega, m)
    A_1_mat = create_A_1_mat(A_mat)
    X_A = funcX(A_1_mat)
    n = A_1_mat.shape[0] + 1
    o_1 = get_o_1(X_A, n)
    print("m = {m_term}, n = {n_term}, χ(A) = {X_A_term}, o(1) term = {o_1_term:.3f}".format(m_term=m, n_term=n, X_A_term=X_A, o_1_term=o_1))

m = 2, n = 4, χ(A) = 1.0, o(1) term = 0.000
m = 3, n = 8, χ(A) = 3.0, o(1) term = 0.302
m = 4, n = 16, χ(A) = 4.404958677685952, o(1) term = 0.866
m = 5, n = 32, χ(A) = 21.002176502710444, o(1) term = 1.363
m = 6, n = 64, χ(A) = 2.5193310468873387, o(1) term = 1.979


m = 7, n = 128, χ(A) = 1.570948438799066, o(1) term = 2.495
m = 8, n = 256, χ(A) = 1.6801286261165949, o(1) term = 2.997
m = 9, n = 512, χ(A) = 4.816800315108802, o(1) term = 3.496
m = 10, n = 1024, χ(A) = 46.76245135942249, o(1) term = 3.995
m = 11, n = 2048, χ(A) = 13.225766859909191, o(1) term = 4.498


### Generate matrix $C$

In [63]:
def square_operator(S, T):
    top_right = np.zeros((S.shape[0], T.shape[1]),dtype=int)
    btn_left = np.zeros((T.shape[0], S.shape[1]),dtype=int)
    if btn_left.shape[1] > 0:
        btn_left[0, -1] = 1
    return np.asarray(np.bmat([[S, top_right], [btn_left, T]]))

res = None

for m in range(2, 5):
    omega = create_Omega(m)
    assert_omega(omega, m)
    A_mat = create_A(omega, m)
    A_1_mat = create_A_1_mat(A_mat)
    res = A_1_mat if res is None else square_operator(res, A_1_mat)

pretty_print(res.astype(int))

┌───────────────────────────────────────────────────┐
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 1 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 1 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 0

In [59]:
S = np.array([[1, 2, 3], 
              [4, 5, 6]])    
T = np.array([[7, 8], 
              [9, 10]])
square_operator(S, T)

array([[ 1,  2,  3,  0,  0],
       [ 4,  5,  6,  0,  0],
       [ 0,  0,  1,  7,  8],
       [ 0,  0,  0,  9, 10]])